## Problem Definition and Goal Setting

**Goal to achieve**: Predict PD progression (MDS-UPDRS) using motor data.

**Smart Goals**:
- **Specific**: Develop an ML model to predict the progression of PD (MDS-UPDRS) using unimodal motor data. 
- **Measurable**: Evaluate the model's performance using appropriate regression metrics such as Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and R-squared (R²). 
- **Achievable**: Use PPMI data.

**Determine type of ML problem**: *Regression*; the task is to predict the continuous progression of motor symptoms over time.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/Users/larsheijnen/Thesis/data/MDS-UPDRS_Part_III_21Mar2025.csv')

In [ ]:
motor_features = [
    'PATNO', 'EVENT_ID', 'INFODT',  # Identifying information
    
    # MDS-UPDRS Part III scores
    'NP3SPCH', 'NP3FACXP', 
    'NP3RIGN', 'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL',  # Rigidity
    'NP3FTAPR', 'NP3FTAPL',  # Finger tapping
    'NP3HMOVR', 'NP3HMOVL',  # Hand movements
    'NP3PRSPR', 'NP3PRSPL',  # Pronation-supination
    'NP3TTAPR', 'NP3TTAPL',  # Toe tapping
    'NP3LGAGR', 'NP3LGAGL',  # Leg agility
    'NP3RISNG',  # Arising from chair
    'NP3GAIT', 'NP3FRZGT',  # Gait and freezing
    'NP3PSTBL',  # Postural stability
    'NP3POSTR',  # Posture
    'NP3BRADY',  # Body bradykinesia
    'NP3PTRMR', 'NP3PTRML',  # Postural tremor
    'NP3KTRMR', 'NP3KTRML',  # Kinetic tremor
    'NP3RTARU', 'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON',  # Rest tremor
    'NP3TOT'  # Total score
]

# Create the DataFrame with selected features (only keep columns that exist in the data)
motor_df = data[[col for col in motor_features if col in data.columns]]

# Display the first few rows to verify the contents
print(f"Motor assessment DataFrame shape: {motor_df.shape}")
motor_df.head()

In [ ]:
missing_percentage = motor_df.isnull().mean() * 100
missing_percentage_sorted = missing_percentage.sort_values(ascending=False)

for column, percentage in missing_percentage_sorted.items():
    print(f"{column}: {percentage:.2f}% missing")

plt.figure(figsize=(14, 6))
missing_percentage_sorted.plot(kind='bar', color='coral')
plt.title('Percentage of Missing Values by Feature', fontsize=14)
plt.xlabel('Features', fontsize=12)
plt.ylabel('Missing Values (%)', fontsize=12)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

**Interpretation**: NP3TOT is missing the most. To remove rows with NaN values.

In [ ]:
motor_df = motor_df.dropna(subset=['NP3TOT'])

print(f"Updated DataFrame shape: {motor_df.shape}")
motor_df.head()

In [ ]:
missing_percentage = motor_df.isnull().mean() * 100
missing_percentage_sorted = missing_percentage.sort_values(ascending=False)

for column, percentage in missing_percentage_sorted.items():
    print(f"{column}: {percentage:.2f}% missing")

plt.figure(figsize=(14, 6))
missing_percentage_sorted.plot(kind='bar', color='coral')
plt.title('Percentage of Missing Values by Feature', fontsize=14)
plt.xlabel('Features', fontsize=12)
plt.ylabel('Missing Values (%)', fontsize=12)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

**Interpretation**: After removing rows with NaN in NP3TOT, no other features have missing values left!

In [ ]:
# Define a function to detect and remove outliers using the IQR method
def remove_outliers_iqr(df, column_name):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

# Apply the function to the 'NP3TOT' column
motor_df_no_outliers = remove_outliers_iqr(motor_df, column)

print(f"Original DataFrame shape: {motor_df.shape}")
print(f"DataFrame shape after removing outliers: {motor_df_no_outliers.shape}")
motor_df_no_outliers.head()

**Interpretation**: Removing outliers removed 218 rows. 

In [ ]:
from sklearn.preprocessing import StandardScaler

motor_df_standardized = motor_df_no_outliers.copy()

# Identify columns to scale: we exclude identifiers and date fields
columns_to_scale = [col for col in motor_df_no_outliers.columns if col not in ['PATNO', 'EVENT_ID', 'INFODT']]

scaler = StandardScaler()
motor_df_standardized[columns_to_scale] = scaler.fit_transform(motor_df_standardized[columns_to_scale])

motor_df_standardized.head()

In [ ]:
from scipy import stats
import seaborn as sns

# Calculate skewness for each numeric feature
skewness = motor_df_standardized.select_dtypes(include=['float64', 'int64']).apply(lambda x: stats.skew(x))
skewness_sorted = skewness.sort_values(ascending=False)

# Display top skewed features
print("Top 10 skewed features:")
print(skewness_sorted.head(10))
print("\nLeast skewed features:")
print(skewness_sorted.tail(10))

# Create a figure to visualize distributions
plt.figure(figsize=(15, 10))

# Plot histogram of NP3TOT (target variable)
plt.subplot(2, 2, 1)
sns.histplot(motor_df_standardized['NP3TOT'], kde=True)
plt.title(f"Distribution of NP3TOT (Skewness: {skewness['NP3TOT']:.3f})")
plt.xlabel('NP3TOT Score')

# Q-Q plot for NP3TOT
plt.subplot(2, 2, 2)
stats.probplot(motor_df_standardized['NP3TOT'], plot=plt)
plt.title("Q-Q Plot of NP3TOT")

# Boxplot for NP3TOT
plt.subplot(2, 2, 3)
sns.boxplot(x=motor_df_standardized['NP3TOT'])
plt.title("Boxplot of NP3TOT")

# Plot distribution of a highly skewed feature
most_skewed_feature = skewness_sorted.index[0]
plt.subplot(2, 2, 4)
sns.histplot(motor_df_standardized[most_skewed_feature], kde=True)
plt.title(f"Distribution of {most_skewed_feature} (Skewness: {skewness[most_skewed_feature]:.3f})")

plt.tight_layout()
plt.show()

In [ ]:
motor_df_standardized['INFODT'] = pd.to_datetime(motor_df_standardized['INFODT'])
motor_df_standardized = motor_df_standardized.sort_values(['PATNO', 'INFODT'])
motor_df_standardized['days_since_baseline'] = motor_df_standardized.groupby('PATNO')['INFODT'] \
    .transform(lambda x: (x - x.min()).dt.days)

motor_df_standardized.head()

In [ ]:
X = motor_df_standardized.drop(['NP3TOT', 'PATNO', 'EVENT_ID', 'INFODT', 'days_since_baseline'], axis=1)
y = motor_df_standardized['NP3TOT']

motor_df_standardized.to_csv('motor_df_standardized.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import csv

# Define models to train
models = {
    "Linear Regression": LinearRegression(),
    "SVR": SVR(),
    "GradientBoosting Regressor": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "KNeighbors": KNeighborsRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
}

# Initialize results list
results = []

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results.append({"Model": name, "MAE": mae, "RMSE": rmse, "R-squared": r2})
    
    # Plot actual vs predicted
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, alpha=0.7)
    plt.xlabel("Actual NP3TOT")
    plt.ylabel("Predicted NP3TOT")
    plt.title(f"Actual vs Predicted NP3TOT ({name})")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.tight_layout()
    plt.show()

# Write results to a CSV file
csv_file = "model_results.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["Model", "MAE", "RMSE", "R-squared"])
    writer.writeheader()
    writer.writerows(results)

print(f"Results written to {csv_file}")

## Cross-Validation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
import matplotlib.pyplot as plt
import numpy as np
import csv

# Define models to train
models = {
    "Linear Regression": LinearRegression(),
    "SVR": SVR(),
    "GradientBoosting Regressor": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "KNeighbors": KNeighborsRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
}

# Initialize results list
results = []

# Define cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate each model using cross-validation
for name, model in models.items():
    print(f"Evaluating {name}...")

    # Cross-validated predictions (for metrics and plots)
    y_pred = cross_val_predict(model, X_train, y_train, cv=cv)

    mae = mean_absolute_error(y_train, y_pred)
    rmse = np.sqrt(mean_squared_error(y_train, y_pred))
    r2 = r2_score(y_train, y_pred)

    results.append({"Model": name, "MAE": mae, "RMSE": rmse, "R-squared": r2})

    # Plot actual vs predicted
    plt.figure(figsize=(8, 6))
    plt.scatter(y_train, y_pred, alpha=0.7)
    plt.xlabel("Actual NP3TOT")
    plt.ylabel("Predicted NP3TOT")
    plt.title(f"Cross-Validated Actual vs Predicted NP3TOT ({name})")
    plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')
    plt.tight_layout()
    plt.show()

# Write results to a CSV file
csv_file = "model_results_cv.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["Model", "MAE", "RMSE", "R-squared"])
    writer.writeheader()
    writer.writerows(results)

print(f"Cross-validated results written to {csv_file}")